In [1]:

from pathlib import Path
from typing import NamedTuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

import gymnasium as gym
from gymnasium.envs.toy_text.frozen_lake import generate_random_map


sns.set_theme()

# %load_ext lab_black

In [2]:
# Parameters

# import gym

# Define the parameters using NamedTuple
class FrozenLakeParams(NamedTuple):
    total_episodes: int  # Total episodes
    learning_rate: float  # Learning rate
    gamma: float  # Discounting rate
    epsilon: float  # Exploration probability
    map_size: int  # Number of tiles of one side of the squared environment
    seed: int  # Define a seed so that we get reproducible results
    is_slippery: bool  # If true the player will move in intended direction with probability of 1/3 else will move in either perpendicular direction with equal probability of 1/3 in both directions
    n_runs: int  # Number of runs
    action_size: int  # Number of possible actions
    state_size: int  # Number of possible states
    proba_frozen: float  # Probability that a tile is frozen
    savefig_folder: Path  # Root folder where plots are saved


params = FrozenLakeParams(
    total_episodes=2000,
    learning_rate=0.8,
    gamma=0.95,
    epsilon=0.1,
    map_size=5,
    seed=123,
    is_slippery=False,
    n_runs=20,
    action_size=None,
    state_size=None,
    proba_frozen=0.9,
    savefig_folder=Path("./img/"),  
)
params

# Set the seed
rng = np.random.default_rng(params.seed)

# Create the figure folder if it doesn't exist
params.savefig_folder.mkdir(parents=True, exist_ok=True)

In [3]:
# Frozen Lake env

# # Create the Frozen Lake environment
# env = gym.make(
#     "FrozenLake-v1",
#     desc=None,  # The description of the map will be generated automatically
#     map_name=None,
#     is_slippery=params.is_slippery,
#     render_mode="human",  # Change to "rgb_array" if you want to visualize the environment
# )

# Update the params object with action_size and state_size
env = gym.make("FrozenLake-v1", is_slippery=params.is_slippery)
params = params._replace(action_size=env.action_space.n)
params = params._replace(state_size=env.observation_space.n)
env.close()  # Don't forget to close the environment after getting the sizes


In [4]:
# Creating the Q-Table

class QLearningTable:
    def __init__(self, gamma, learning_rate, state_size, action_size):
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.state_size = state_size
        self.action_size = action_size
        self.q_table = np.zeros((state_size, action_size))

    def reset_qtable(self):
        self.q_table = np.zeros((self.state_size, self.action_size))

    def update(self, state, action, reward, new_state):
        predict = self.q_table[state, action]
        target = reward + self.gamma * np.max(self.q_table[new_state, :])
        self.q_table[state, action] += self.learning_rate * (target - predict)


# class EpsilonGreedyPolicy:
#     def __init__(self, epsilon, rng=None):
#         self.epsilon = epsilon
#         self.rng = rng

#     def choose_action(self, action_space, state, q_table):
#         """Choose an action `a` in the current world state (s)."""
#         if self.rng is not None:
#             # First, we randomize a number
#             explor_exploit_tradeoff = self.rng.uniform(0, 1)

#             # Exploration
#             if explor_exploit_tradeoff < self.epsilon:
#                 action = action_space.sample()

#             # Exploitation (taking the biggest Q-value for this state)
#             else:
#                 # Break ties randomly
#                 # If all actions are the same for this state, we choose a random one
#                 # (otherwise `np.argmax()` would always take the first one)
#                 if np.all(q_table[state, :] == q_table[state, 0]):
#                     action = action_space.sample()
#                 else:
#                     action = np.argmax(q_table[state, :])
#         else:
#             # Original implementation without RNG
#             if np.random.rand() < self.epsilon:
#                 action = action_space.sample()
#             else:
#                 max_q_value = np.max(q_table[state, :])
#                 action = np.random.choice(np.where(q_table[state, :] == max_q_value)[0])
#         return action




# # Modify the EpsilonGreedyPolicy class to use the random number generator
# class EpsilonGreedyPolicy:
#     def __init__(self, epsilon, rng):
#         self.epsilon = epsilon
#         self.rng = rng

#     def choose_action(self, action_space, state, q_table):
#         if self.rng.random() < self.epsilon:
#             action = action_space.sample()
#         else:
#             max_q_value = np.max(q_table[state, :])
#             argmax_actions = np.where(q_table[state, :] == max_q_value)[0]
#             action = self.rng.choice(argmax_actions)
#         return action



class EpsilonGreedyPolicy:
    def __init__(self, epsilon):
        self.epsilon = epsilon

    def choose_action(self, action_space, state, q_table):
        """Choose an action `a` in the current world state (s)."""
        # First, we randomize a number
        explor_exploit_tradeoff = rng.uniform(0, 1)

        # Exploration
        if explor_exploit_tradeoff < self.epsilon:
            action = action_space.sample()

        # Exploitation (taking the biggest Q-value for this state)
        else:
            # Break ties randomly
            # If all actions are the same for this state, we choose a random one
            # (otherwise `np.argmax()` would always take the first one)
            if np.all(q_table[state, :] == q_table[state, 0]):
                action = action_space.sample()
            else:
                action = np.argmax(q_table[state, :])
        return action


In [5]:
# learner = Qlearning(
#     learning_rate=params.learning_rate,
#     gamma=params.gamma,
#     state_size=params.state_size,
#     action_size=params.action_size,
# )
# explorer = EpsilonGreedy(
#     epsilon=params.epsilon,
# )

In [6]:
def run_env(params):
    env = gym.make("FrozenLake-v1", is_slippery=params.is_slippery)
    learner = QLearningTable(
        gamma=params.gamma,
        learning_rate=params.learning_rate,
        state_size=params.state_size,
        action_size=params.action_size,
    )
    explorer = EpsilonGreedyPolicy(params.epsilon, rng=np.random.default_rng(params.seed))

    rewards = np.zeros((params.total_episodes, params.n_runs))
    steps = np.zeros((params.total_episodes, params.n_runs))
    episodes = np.arange(params.total_episodes)
    qtables = np.zeros((params.n_runs, params.state_size, params.action_size))
    all_states = []
    all_actions = []

    for run in range(params.n_runs):  # Run several times to account for stochasticity
        learner.reset_qtable()  # Reset the Q-table between runs

        for episode in tqdm(episodes, desc=f"Run {run+1}/{params.n_runs} - Episodes", leave=False):
            state = env.reset()  # Reset the environment
            step = 0
            done = False
            total_rewards = 0

            while not done:
                action = explorer.choose_action(action_space=env.action_space, state=state, q_table=learner.q_table)

                # Log all states and actions
                all_states.append(state.item())  # Convert state to a scalar integer
                all_actions.append(action)

                # Take the action (a) and observe the outcome state(s') and reward (r)
                new_state, reward, done, _ = env.step(action)

                learner.update(state.item(), action, reward, new_state.item())

                total_rewards += reward
                step += 1

                # Our new state is state
                state = new_state

            # Log all rewards and steps
            rewards[episode, run] = total_rewards
            steps[episode, run] = step

        qtables[run, :, :] = learner.q_table

    env.close()  # Close the environment when done
    return rewards, steps, episodes, qtables, all_states, all_actions




In [7]:
# Visualization

def postprocess(episodes, params, rewards, steps, map_size):
    """Convert the results of the simulation in dataframes."""
    res = pd.DataFrame(
        data={
            "Episodes": np.tile(episodes, reps=params.n_runs),
            "Rewards": rewards.flatten(),
            "Steps": steps.flatten(),
        }
    )
    res["cum_rewards"] = rewards.cumsum(axis=0).flatten(order="F")
    res["map_size"] = np.repeat(f"{map_size}x{map_size}", res.shape[0])

    st = pd.DataFrame(data={"Episodes": episodes, "Steps": steps.mean(axis=1)})
    st["map_size"] = np.repeat(f"{map_size}x{map_size}", st.shape[0])
    return res, st

In [8]:
def qtable_directions_map(qtable, map_size):
    """Get the best learned action & map it to arrows."""
    qtable_val_max = qtable.max(axis=1).reshape(map_size, map_size)
    qtable_best_action = np.argmax(qtable, axis=1).reshape(map_size, map_size)
    directions = {0: "←", 1: "↓", 2: "→", 3: "↑"}
    qtable_directions = np.empty(qtable_best_action.flatten().shape, dtype=str)
    eps = np.finfo(float).eps  # Minimum float number on the machine
    for idx, val in enumerate(qtable_best_action.flatten()):
        if qtable_val_max.flatten()[idx] > eps:
            # Assign an arrow only if a minimal Q-value has been learned as best action
            # otherwise since 0 is a direction, it also gets mapped on the tiles where
            # it didn't actually learn anything
            qtable_directions[idx] = directions[val]
    qtable_directions = qtable_directions.reshape(map_size, map_size)
    return qtable_val_max, qtable_directions

In [9]:
def plot_q_values_map(qtable, env, map_size):
    """Plot the last frame of the simulation and the policy learned."""
    qtable_val_max, qtable_directions = qtable_directions_map(qtable, map_size)

    # Plot the last frame
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    ax[0].imshow(env.render())
    ax[0].axis("off")
    ax[0].set_title("Last frame")

    # Plot the policy
    sns.heatmap(
        qtable_val_max,
        annot=qtable_directions,
        fmt="",
        ax=ax[1],
        cmap=sns.color_palette("Blues", as_cmap=True),
        linewidths=0.7,
        linecolor="black",
        xticklabels=[],
        yticklabels=[],
        annot_kws={"fontsize": "xx-large"},
    ).set(title="Learned Q-values\nArrows represent best action")
    for _, spine in ax[1].spines.items():
        spine.set_visible(True)
        spine.set_linewidth(0.7)
        spine.set_color("black")
    img_title = f"frozenlake_q_values_{map_size}x{map_size}.png"
    fig.savefig(params.savefig_folder / img_title, bbox_inches="tight")
    plt.show()

In [10]:
def plot_states_actions_distribution(states, actions, map_size):
    """Plot the distributions of states and actions."""
    labels = {"LEFT": 0, "DOWN": 1, "RIGHT": 2, "UP": 3}

    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    sns.histplot(data=states, ax=ax[0], kde=True)
    ax[0].set_title("States")
    sns.histplot(data=actions, ax=ax[1])
    ax[1].set_xticks(list(labels.values()), labels=labels.keys())
    ax[1].set_title("Actions")
    fig.tight_layout()
    img_title = f"frozenlake_states_actions_distrib_{map_size}x{map_size}.png"
    fig.savefig(params.savefig_folder / img_title, bbox_inches="tight")
    plt.show()

In [11]:
map_sizes = [5, 8, 11, 14]
res_all = pd.DataFrame()
st_all = pd.DataFrame()

for map_size in map_sizes:
    env = gym.make(
        "FrozenLake-v1",
        is_slippery=params.is_slippery,
        render_mode="rgb_array",
        desc=generate_random_map(
            size=map_size, p=params.proba_frozen, seed=params.seed
        ),
    )

    params = params._replace(action_size=env.action_space.n)
    params = params._replace(state_size=env.observation_space.n)
    env.action_space.seed(
        params.seed
    )  # Set the seed to get reproducible results when sampling the action space
    learner = QLearningTable(
        learning_rate=params.learning_rate,
        gamma=params.gamma,
        state_size=params.state_size,
        action_size=params.action_size,
    )
    explorer = EpsilonGreedyPolicy(
        epsilon=params.epsilon,
        rng=np.random.default_rng(params.seed)  # Provide the rng argument
    )

    print(f"Map size: {map_size}x{map_size}")
    rewards, steps, episodes, qtables, all_states, all_actions = run_env(params)

    # Save the results in dataframes
    res, st = postprocess(episodes, params, rewards, steps, map_size)
    res_all = pd.concat([res_all, res])
    st_all = pd.concat([st_all, st])
    qtable = qtables.mean(axis=0)  # Average the Q-table between runs

    plot_states_actions_distribution(
        states=all_states, actions=all_actions, map_size=map_size
    )  # Sanity check
    plot_q_values_map(qtable, env, map_size)

    env.close()


TypeError: EpsilonGreedyPolicy.__init__() got an unexpected keyword argument 'rng'

In [ ]:
def plot_steps_and_rewards(rewards_df, steps_df):
    """Plot the steps and rewards from dataframes."""
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    sns.lineplot(
        data=rewards_df, x="Episodes", y="cum_rewards", hue="map_size", ax=ax[0]
    )
    ax[0].set(ylabel="Cumulated rewards")

    sns.lineplot(data=steps_df, x="Episodes", y="Steps", hue="map_size", ax=ax[1])
    ax[1].set(ylabel="Averaged steps number")

    for axi in ax:
        axi.legend(title="map size")
    fig.tight_layout()
    img_title = "frozenlake_steps_and_rewards.png"
    fig.savefig(params.savefig_folder / img_title, bbox_inches="tight")
    plt.show()


plot_steps_and_rewards(res_all, st_all)